In [ ]:
# Dataset to train the model with numbers in sign language
# It's a Kaggle dataset available at the following link:
# https://www.kaggle.com/datasets/ardamavi/sign-language-digits-dataset
# It's uploaded to a git repository, so we'll clone it directly
!git clone https://github.com/ardamavi/Sign-Language-Digits-Dataset.git

In [ ]:
# We import all the necessary packages.
import tensorflow as tf

from tensorflow.keras.models                import Sequential, Model
from tensorflow.keras.layers                import Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image   import ImageDataGenerator
from tensorflow.keras.optimizers            import SGD
from tensorflow.keras                       import regularizers

# Callbacks
from tensorflow.keras.callbacks             import ModelCheckpoint
from tensorflow.keras.callbacks             import ReduceLROnPlateau

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.preprocessing.image   import ImageDataGenerator

In [53]:
# Variables for the model operation
clases = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
num_classes = len(clases) # There are a total of 10 classes.
epochs = 50
bs = 32 # Bachsize
k = 2

In [ ]:
# We create image generators for the train and test/val datasets
# ImageDataGenerator is a Keras class that allows data augmentation
# for image data in real-time during training.

# Training image generator.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=(0.3),
        zoom_range=(0.3),
        width_shift_range=(0.2),
        height_shift_range=(0.2),
        validation_split = 0.2,
        brightness_range=(0.05,0.85),
        horizontal_flip=False)

# Loading images into the training generator from directory.
train_generator = train_datagen.flow_from_directory(
        '/content/Sign-Language-Digits-Dataset/Dataset',
        class_mode='categorical',
        shuffle=True,
        target_size=(28*k, 28*k),
        color_mode = 'rgb',
        subset = 'training',
        batch_size=bs)

valid_generator = train_datagen.flow_from_directory(
        '/content/Sign-Language-Digits-Dataset/Dataset',
        class_mode='categorical',
        shuffle=True,
        target_size=(28*k, 28*k),
        color_mode = 'rgb',
        subset = 'validation',
        batch_size=bs)

Found 1653 images belonging to 10 classes.
Found 409 images belonging to 10 classes.


In [ ]:
# We import the model we are going to use, which is VGG19.
VGG19_model = tf.keras.applications.VGG19(input_shape=(28*k,28*k,3),
                                          include_top=False,
                                          weights='imagenet')

# We display the layers of this model.
VGG19_model.summary()

In [ ]:
# Freezing the first layers of a convolutional model, such as VGG19, is because
# they generally learn basic and generic features, such as edges,
# simple textures, etc. These features are useful for a wide range of
# computer vision tasks and are often transferable to different datasets
# and problems.

# We freeze the first 6 layers of the model to proceed to train the rest.
for layer in VGG19_model.layers[:6]:
  layer.trainable = False

In [ ]:
# We create a new empty model.
model = tf.keras.Sequential()

# We add the pre-trained model as if it were a layer.
model.add(VGG19_model)

# We continue adding more layers that will be trained.
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01), activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation = 'softmax'))

In [ ]:
# We compile the model.
model.compile(loss="categorical_crossentropy",
              optimizer= SGD(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
# This callback is used to save the model during training.
# You can specify various parameters, such as the file name to
# save the model to, whether to save only the best model or all models.
checkpointer = ModelCheckpoint(filepath='model', verbose=1, save_best_only=True,
                               monitor = 'val_acc', mode = 'max')

# This callback is used to dynamically adjust the learning rate
# during training. Reducing the learning rate can help prevent
# the model from getting stuck in a local minimum during training.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.000001)

In [ ]:
# We train the model.
history= model.fit(train_generator,
                   validation_data = valid_generator,
                   callbacks = [reduce_lr, checkpointer],
                   epochs=epochs)

In [ ]:
# Save trained model.
model.save('model_sign_language.keras')

In [ ]:
# Accuracy plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','test'])
plt.show()

# Loss plot
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title("Loss")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['val_loss','loss'])
plt.show()

In [ ]:
import os
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg19 import preprocess_input
from google.colab import files

# Uncomment code if you want to upload image from your PC
#uploaded=files.upload()
#file=list(uploaded.keys())[0]
#path='/content/' + file

# Path to one of the example images, ranging from example_0 to example_9
path = '/content/Sign-Language-Digits-Dataset/Examples/example_3.JPG'

# Load the image and resize it to (28*k, 28*k) as expected by the model
img = image.load_img(path, target_size=(28*k, 28*k))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Add an additional dimension to represent the batch
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image (normalize, etc.)
img_array = preprocess_input(img_array)

# Make the prediction
predicted_batch = model.predict(img_array)

# Get the predicted classes
predicted_class = np.argmax(predicted_batch)

# Get the name of the predicted class
predicted_label = clases[predicted_class]

# Show the image with the prediction
plt.imshow(img)
plt.title('Predicted Class: ' + predicted_label)
plt.show()
